In [1]:
import requests
import json
import os
from time import sleep
API_KEY = {'X-API-Key': 'KZJ94OPT'}
session = requests.Session()
session.headers.update(API_KEY)

In [2]:
class ApiException(Exception):
    pass

class HelperFunctions():
    
    def __init__(self):
        pass

    def ticker_bid_ask(self, session, ticker):
        payload = {'ticker': ticker}
        resp = session.get('http://localhost:9999/v1/securities/book', params=payload)  
        if resp.ok:
            book = resp.json()
            return book['bids'][0]['price'], book['asks'][0]['price']
        raise ApiException('Error getting bid / ask: The API key provided in this Python code must match that in the RIT client')

In [32]:
def get_tick(session):
    resp = session.get('http://localhost:9999/v1/case')
    if resp.status_code == 401:
        raise ApiException('Error getting tick: The API key provided in this Python code must match that in the RIT client')
    case = resp.json()
    return case['tick']

In [11]:
assets = session.get('http://localhost:9999/v1/assets')
assets.json()

[{'ticker': 'CL-STORAGE',
  'type': 'CONTAINER',
  'description': 'Storage for Crude Oil Spot in Cushing',
  'total_quantity': 1000,
  'available_quantity': 1000,
  'lease_price': 500.0,
  'convert_from': [],
  'convert_to': [],
  'containment': {'ticker': 'CL', 'quantity': 10.0},
  'ticks_per_conversion': 0,
  'ticks_per_lease': 30,
  'is_available': True,
  'start_period': 1,
  'stop_period': 2},
 {'ticker': 'AK-STORAGE',
  'type': 'CONTAINER',
  'description': 'Storage for Crude Oil in Alaska',
  'total_quantity': 1000,
  'available_quantity': 1000,
  'lease_price': 500.0,
  'convert_from': [],
  'convert_to': [],
  'containment': {'ticker': 'CL-AK', 'quantity': 10.0},
  'ticks_per_conversion': 0,
  'ticks_per_lease': 30,
  'is_available': True,
  'start_period': 1,
  'stop_period': 2},
 {'ticker': 'NYC-STORAGE',
  'type': 'CONTAINER',
  'description': 'Storage for Crude Oil in New York City',
  'total_quantity': 1000,
  'available_quantity': 1000,
  'lease_price': 500.0,
  'convert

In [114]:
# RENT STORAGE
for i in range(3):
    session.post('http://localhost:9999/v1/leases', params = {'ticker': 'CL-STORAGE', 'from':'CONTAINER', 'quantity': 1})

# BUY OIL
session.post('http://localhost:9999/v1/orders', params = {'ticker': 'CL', 'type': 'MARKET', 'quantity': 30, 'action': 'BUY'})

# OIL FUTURE
securities = session.get('http://localhost:9999/v1/securities')
oil_future = securities.json()[3]['ticker']
print(oil_future)

if oil_future == 'CL-1F':
    session.post('http://localhost:9999/v1/orders', params = {'ticker': 'CL-1F', 'type': 'MARKET', 'quantity': 30, 'action': 'SELL'})
else:
    session.post('http://localhost:9999/v1/orders', params = {'ticker': 'CL-2F', 'type': 'MARKET', 'quantity': 30, 'action': 'SELL'})

CL-1F


In [35]:
x = session.get('http://localhost:9999/v1/securities')
x.json()[0]

{'ticker': 'CL',
 'type': 'SPOT',
 'size': 1000,
 'position': 30.0,
 'vwap': 97.3,
 'nlv': 2898600.0,
 'last': 96.63,
 'bid': 96.62,
 'bid_size': 300000.0,
 'ask': 96.64,
 'ask_size': 300000.0,
 'volume': 30.0,
 'unrealized': -20400.0,
 'realized': -30.0,
 'currency': '',
 'total_volume': 30.0,
 'limits': [{'name': 'CRUDE', 'units': 1.0}],
 'interest_rate': 0.0,
 'is_tradeable': True,
 'is_shortable': False,
 'start_period': 1,
 'stop_period': 2,
 'description': 'Crude Oil Spot',
 'unit_multiplier': 1000,
 'display_unit': 'Barrels',
 'start_price': 100.0,
 'min_price': 0.0,
 'max_price': 1000.0,
 'quoted_decimals': 2,
 'trading_fee': 1.0,
 'limit_order_rebate': 0.0,
 'min_trade_size': 0,
 'max_trade_size': 30,
 'required_tickers': 'CL-STORAGE',
 'underlying_tickers': None,
 'bond_coupon': 0.0,
 'interest_payments_per_period': 0,
 'base_security': '',
 'fixing_ticker': None,
 'api_orders_per_second': 100,
 'execution_delay_ms': 0,
 'interest_rate_ticker': None,
 'otc_price_range': 0.0}

In [115]:
session.post('http://localhost:9999/v1/leases', params = {'ticker': 'CL-REFINERY', 'from':'REFINERY', 'quantity': 1})



<Response [200]>

In [153]:
# USE REFINERY
leases = session.get('http://localhost:9999/v1/leases')

for i in leases.json():
    if i['ticker'] == 'CL-REFINERY':
        refinery_id = i['id']
print(refinery_id)

# session.get('http://localhost:9999/v1/leases/{}', params = {'ticker': 'CL-REFINERY', 'from': 'CL', 'quantity': 30, 'id': refinery_id})

# session.get('http://localhost:9999/v1/leases/{}'.format(refinery_id))

session.get('http://localhost:9999/v1/leases/{}'.format(refinery_id), params = {'ticker': 'CL-REFINERY', 'from1': 'CL', 'quantity1': 30})

7


<Response [200]>

In [164]:
# GET RB AND HO POSITIONS
ho_position = 0
rb_position = 0

while ho_position == 0 or rb_position == 0:
    ho = session.get('http://localhost:9999/v1/securities', params={'ticker': 'HO'})
    if ho.json()[0]['position'] != 0.0:
        ho_position = ho.json()[0]['position']
    rb = session.get('http://localhost:9999/v1/securities', params={'ticker': 'RB'})
    if rb.json()[0]['position'] != 0.0:
        rb_position = rb.json()[0]['position']

# SELL HO, RB, AND FUTURES CONTRACT
cl_1f = session.get('http://localhost:9999/v1/securities', params={'ticker': 'CL-1F'})
cl_1f_position = cl_1f.json()[0]['position']
cl_2f = session.get('http://localhost:9999/v1/securities', params={'ticker': 'CL-2F'})
cl_2f_position = cl_2f.json()[0]['position']

if cl_1f_position < 0.0:
    session.post('http://localhost:9999/v1/orders', params={'ticker': 'CL-1F', 'type': 'MARKET', 'quantity': -cl_1f_position, 'action': 'BUY'})
if cl_2f_position < 0.0:
    session.post('http://localhost:9999/v1/orders', params={'ticker': 'CL-2F', 'type': 'MARKET', 'quantity': -cl_2f_position, 'action': 'BUY'})

# SELL RB AND HO
session.post('http://localhost:9999/v1/orders', params={'ticker': 'RB', 'type': 'MARKET', 'quantity': rb_position, 'action': 'SELL'})
session.post('http://localhost:9999/v1/orders', params={'ticker': 'HO', 'type': 'MARKET', 'quantity': ho_position, 'action': 'SELL'})


In [156]:
ho = session.get('http://localhost:9999/v1/securities', params={'ticker': 'HO'})
ho.json()

[{'ticker': 'HO',
  'type': 'SPOT',
  'size': 42000,
  'position': 0.0,
  'vwap': 0.0,
  'nlv': 0.0,
  'last': 3.97,
  'bid': 3.96,
  'bid_size': 300000.0,
  'ask': 3.98,
  'ask_size': 300000.0,
  'volume': 0.0,
  'unrealized': 0.0,
  'realized': 0.0,
  'currency': '',
  'total_volume': 0.0,
  'limits': [{'name': 'PRODUCT', 'units': 1.0}],
  'interest_rate': 0.0,
  'is_tradeable': True,
  'is_shortable': False,
  'start_period': 1,
  'stop_period': 2,
  'description': 'Heating Oil',
  'unit_multiplier': 42000,
  'display_unit': 'Gallons',
  'start_price': 3.0,
  'min_price': 0.0,
  'max_price': 1000.0,
  'quoted_decimals': 2,
  'trading_fee': 1.0,
  'limit_order_rebate': 0.0,
  'min_trade_size': 0,
  'max_trade_size': 30,
  'required_tickers': None,
  'underlying_tickers': None,
  'bond_coupon': 0.0,
  'interest_payments_per_period': 0,
  'base_security': '',
  'fixing_ticker': None,
  'api_orders_per_second': 100,
  'execution_delay_ms': 0,
  'interest_rate_ticker': None,
  'otc_price

In [ ]:
def get_position(session, ticker):
    positions = session.get('http://localhost:9999/v1/securities', params={'ticker': ticker})
    position = positions.json()[0]['position']
    return position

In [112]:
assets = session.get('http://localhost:9999/v1/assets/history', params = {'ticker': 'CL-REFINERY', 'limit': 100})
assets.json()

[{'tick': 16,
  'ticker': 'CL-REFINERY',
  'action': 'LEASE',
  'cost': 300000.0,
  'convert_from': None,
  'convert_to': None,
  'convert_from_price': None,
  'convert_to_price': None}]

In [67]:
securities.json()

[{'ticker': 'CL',
  'type': 'SPOT',
  'size': 1000,
  'position': 0.0,
  'vwap': 0.0,
  'nlv': 0.0,
  'last': 102.32,
  'bid': 102.31,
  'bid_size': 300000.0,
  'ask': 102.33,
  'ask_size': 300000.0,
  'volume': 60.0,
  'unrealized': 0.0,
  'realized': -7860.0,
  'currency': '',
  'total_volume': 60.0,
  'limits': [{'name': 'CRUDE', 'units': 1.0}],
  'interest_rate': 0.0,
  'is_tradeable': True,
  'is_shortable': False,
  'start_period': 1,
  'stop_period': 2,
  'description': 'Crude Oil Spot',
  'unit_multiplier': 1000,
  'display_unit': 'Barrels',
  'start_price': 100.0,
  'min_price': 0.0,
  'max_price': 1000.0,
  'quoted_decimals': 2,
  'trading_fee': 1.0,
  'limit_order_rebate': 0.0,
  'min_trade_size': 0,
  'max_trade_size': 30,
  'required_tickers': 'CL-STORAGE',
  'underlying_tickers': None,
  'bond_coupon': 0.0,
  'interest_payments_per_period': 0,
  'base_security': '',
  'fixing_ticker': None,
  'api_orders_per_second': 100,
  'execution_delay_ms': 0,
  'interest_rate_ticker

In [ ]:
session.post('http://localhost:9999/v1/leases', params = {'ticker': 'CL-STORAGE', 'from':'CONTAINER', 'quantity': 1})


buy storage
buy oil
sell CL1F / CL2F -> if statement for time -> add in another clause that only locks in the future if price is going to go up
buy refinery
refine oil
cancel storage
sell HO and RB
buy back 
REMEMBER TO ACCOUNT FOR TRADING LIMIT
make sure to not hold CL futures at the end of periods -> if tick = 598, sell future

# STRATEGY

- pull up 2 windows for assets
- the algo will handle everything except cancelling refining
- once we purchase oil, immediately start refining
- once we start refining, cancel storage